Viinien luokittelu
==============

Tässä harjoituksessa tutustutaan viinien maailmaan.

**Ongelma**: Voiko viinin laadun päätellä viiniä kuvaavien arvojen perusteella?

## Data lataus
Lataa Pandas DataFrame-olioina aineistot "../winequality-red.csv" ja "../winequality-white.csv".

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

def process_data(name, label):
    wine = pd.read_csv(name, index_col=False)
    wine['label'] = label 
    return wine
    
red = process_data("../winequality-red.csv", 0)
white = process_data("../winequality-white.csv", 1)

wine = pd.concat([red, white])
# Sample sekoittaa aineiston, frac=1 tarkoittaa palauttaa koko aineiston
wine.sample(frac=1).head(10)

## Datan esikäsittely
Quality-arvo vaikuttaa siltä, että se ei ala nollasta. Tarkistetaan tilanne. Muuta se alkamaan 0:sta

In [ ]:
wine.quality.unique()

In [ ]:
wine.quality = wine.quality - wine.quality.min()

#### Muuttujien valinta
Muodosta nyt havaintojoukko `X` ja luokkajoukko `y`.
Valitse havaintojoukkoon mieleikkäämmät sarakkeet viinidatasta. Saat valittua vain osan sarakkeista alla olevalla tyylillä:
```python
# Olemassa olevat
print(list(df.columns))
df = df[['vain', 'halutut', 'sarakkeet']]
```

Valitse luokiksi `y` laatusarake `quality`. Muista kommentoida tämä pois `X`:n sarakkeista.

In [ ]:
y = wine['quality']
X = wine[[
 'fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 #'quality',
 'label'
         ]]

#### Jako
Jaa luomasi X ja luokat y opetus- ja testijoukkoihin X_train, X_test, y_train ja  y_test siten, että testijoukon osuus on 30% ja opetusjoukon osuus on 70% havainnoista.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

##### Datan visualisointia
Visualisoidaan opetusjoukkoa hieman ongelman hahmottamiseksi. Koska valitsemiesi muuttujien määrä voi olla enemmän kuin 2, käytetään PCA-analyysiä visualisoinnin tukena. 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_train_scaled = StandardScaler().fit_transform(X_train)
X_train_pca = pca.fit_transform(X_train_scaled)
print("Muuttujien määrä oli ennen {}, nyt se on {}".format(len(X.columns), len(X_train_pca[0])))

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap=plt.cm.Set1,
            edgecolor='k')
ax.set_xlabel('1. ominaisuusvektori')
ax.set_ylabel('2. ominaisuusvektori')
plt.show()

## Luokittelu
Muodosta sitten putki ([`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline)), johon liität StandardScaler ja PCA esikäsittelyvaiheet, käytä PCA:n `n_components`-parametrina arvoa 0.9. Lisää putken viimeiseksi komponentiksi haluamasi luokittelija nimelle "clf". Jos käytät Keras-luokittelijaa, muista luoda funktio sille oletusparametreilla.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', StandardScaler()), ('pca',PCA(n_components=0.9)), ('clf',SVC())])

Suorita sitten hyperparametrioptimointi. Putkitusta käytettäessä laita parametrisanakirjaan luokittelijakomponentin nimi ja kaksi alaviivaa ennen parametrin nimeä. Esimerkiksi SVM-algoritmin `C`-parametrin tapauksessa:
```python
params_without_pipeline = {'C':[1,10]}
params_with_pipeline = {'clf__C':[1,10]}

```

In [ ]:
from sklearn.model_selection import GridSearchCV

def train(pipe, parameters):
    clf = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1)
    clf.fit(X_train, y_train)
    print("Parhaat parametrit: ", clf.best_params_)
    print("Paras opetus OA: {:.4f}".format(clf.best_score_))
    print("OA: {:.4f}".format(clf.score(X_test, y_test)))
    return clf

### Evaluointi
Luo vielä funktio evaluointia varten

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from utils import plot_confusion_matrix

def evaluate(y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print(classification_report(y_test, y_pred))
    plot_confusion_matrix(cm,list(range(y_pred.max()+1)), list(range(y_pred.max()+1)), normalize=True)
evaluate(y_pred)

##### Mallin opettaminen ja evaluointi
Anna toiseksi parametriksi sanakirja hyperparametreista.

In [ ]:
clf = train(pipe, {'clf__C':[1, 10, 100], 'clf__gamma':[1, 0.1, 0.01]})
y_pred = clf.predict(X_test)
evaluate(y_pred)

Kokeile vielä valita eri määrä muuttujia kohdassa [Muuttujien valinta](#Muuttujien-valinta). Aja se solu ja tämän jälkeen solut [Jako](#Jako) ja [Mallin opettaminen ja evaluointi](#Mallin-opettaminen-ja-evaluointi).

## Viinin tyypin luokittelu


**Ongelma**: Voiko viinin tyypin (punaviini / valkoviini) päätellä viiniä kuvaavien arvojen perusteella?

Valitse nyt [Muuttujien valinta](#Muuttujien-valinta)-kohdassa luokiksi `y` sarake "label" ja muista kommentoida se pois joukosta `X`. Aja se solu ja tämän jälkeen solut [Jako](#Jako), [Datan visualisointia](#Datan-visualisointia) ja [Mallin opettaminen ja evaluointi](#Mallin-opettaminen-ja-evaluointi).